In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import MDAnalysis.analysis.base
from MDAnalysis.analysis import distances
import numpy as np

class DistanceAnalysis(MDAnalysis.analysis.base.AnalysisBase):
  def __init__(self, ag1, ag2, **kwargs):
    super(DistanceAnalysis, self).__init__(ag1.universe.trajectory, **kwargs)
    self._ag1 = ag1
    self._ag2 = ag2
    
  def _prepare(self):
    self.result = []
    
  def _single_frame(self):
    self.result.append(distances.dist(self._ag1, self._ag2)[-1])

  def _conclude(self):
    self.result = np.array(self.result)

In [ ]:
# Loads the aligned trajectory
import MDAnalysis as mda
ref = mda.Universe('../sarscov2-10921231-structure/protein_conf.gro')
sim = mda.Universe('../sarscov2-10921231-structure/protein_conf.gro', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_00to02us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_02to04us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_04to06us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_06to08us.xtc', \
                   '../traj_protein_snap_every1ns/protein_snap_every1ns_08to10us.xtc')

In [ ]:
# Select sulfur on Cys 145 and nitrogen on ring of His 41
SG = sim.select_atoms("resnum 145 and name SG")
SG = SG[0] + SG[0] + SG[1] + SG[1]
N = sim.select_atoms("resnum 41 and (name ND1 or name NE2)")

# Calculate distances throughout the trajectory
R = DistanceAnalysis(SG, N)
R.run()

# Calculate minimum distance between SG and N
d_monomerA = np.min(R.result[:,0:1],1)
d_monomerB = np.min(R.result[:,2:3],1)

In [ ]:
# The distance as a function of time
plt.plot(d_monomerA,'.-')
plt.plot(d_monomerB,'.-')

# Probability densities of dyad distance

In [ ]:
[pdf, xlim] = np.histogram(d_monomerA, bins=50, range=(2.5,10), density=True)

plt.figure(0)
plt.plot(xlim[:-1], pdf)
plt.title('Histogram')

RT = 8.314E-3/4.184*300
plt.figure(1)
plt.plot(xlim[:-1], -RT*np.log(pdf))
plt.title('Potential of mean force')

[pdf, xlim] = np.histogram(d_monomerB, bins=50, range=(2.5,10), density=True)

plt.figure(0)
plt.plot(xlim[:-1], pdf)
plt.title('Histogram')

plt.figure(1)
plt.plot(xlim[:-1], -RT*np.log(pdf))
plt.title('Potential of mean force')

In [ ]:
(pdf, xedges, yedges) = np.histogram2d(d_monomerA, d_monomerB, bins=30, range=((2.5,10),(2.5,10)))
X, Y = np.meshgrid(xedges, yedges)

plt.figure(0)
plt.pcolormesh(X,Y,pdf)
plt.colorbar()

plt.figure(1)
F = -RT*np.log(pdf)
F[F==np.inf] = 0
plt.pcolormesh(X,Y,F)
plt.colorbar()

# Joint probabilities of short dyad distances

In [ ]:
d_cutoff = 4.25
pA = np.sum(d_monomerA<d_cutoff)/len(d_monomerA)
pAB = np.sum(np.logical_and(d_monomerA<d_cutoff, d_monomerB<d_cutoff))/len(d_monomerB)
print('The fraction of frames with a short dyad distance')
print('in monomer A is', pA)
print('in monomer A is ...')
print('in both monomers is', pAB)
print('If the monomers were indepedent, the probabilities would be ...')